<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [236]:
import scannerpy
import scannertools as st
from query.models import Video
from esper.prelude import *
import rekall as rk
from rekall.interval_list import IntervalList
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.payload_predicates import payload_satisfies
from esper.rekall import intrvllists_to_result, intrvllists_to_result_with_objects
import numpy as np

In [3]:
# Load flow histograms from Scanner
video = Video.objects.get(id=123)
frames = [list(range(0, video.num_frames))]

db = scannerpy.Database()

flow_histograms = st.histograms.compute_flow_histograms(
    db,
    videos=[video.for_scannertools()],
    frames=frames)

In [213]:
# Load histograms of angles into rekall
angle_hists = rk.interval_list.IntervalList([
    (frame, frame + 1, hist[1])
    for frame, hist in enumerate(flow)
])

In [225]:
# Only keep frames where at least one of the bins (except for the 0 bin) has a large amount of flow
avg = video.width * video.height / 64
non_zero_frames = angle_hists.filter(payload_satisfies(
    lambda hist: np.max(hist[1:]) > 10 * avg
))

In [242]:
def avg_direction(hist):
    # compute the average unit vector
    units = [[h * math.cos(idx * 2 * math.pi / 64),
            h * math.sin(idx * 2 * math.pi / 64)]
            for idx, h in enumerate(hist[1:].tolist())]
    avg_x = np.sum([unit[0] for unit in units]) / np.sum(hist[1:].tolist())
    avg_y = np.sum([unit[1] for unit in units]) / np.sum(hist[1:].tolist())
    
    return math.atan2(avg_y, avg_x)

def same_direction(hist1, hist2):
    # return True if hist1 and hist2 are pointed in roughly the same direction
    hist1_dir = avg_direction(hist1)
    hist2_dir = avg_direction(hist2)
    angle_diff = min((2 * math.pi) - abs(hist1_dir - hist2_dir), abs(hist1_dir - hist2_dir))
    return abs(angle_diff) < 90 * math.pi / 180

In [ ]:
# Coalesce sequences that have similar motion flow directions
panning_sequences = non_zero_frames.coalesce(
    predicate=payload_satisfies(same_direction, arity=2)
).filter_length(min_length=24)

In [244]:
esper_widget(intrvllists_to_result_with_objects(
    {123: panning_sequences.map(lambda intrvl: (intrvl.start, intrvl.end, 0))},
    lambda payload, video: []
), show_middle_frame=False)

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Video', ['id', 'path', 'num_frames', 'fps', …